# 0. Import required packages

In [ ]:
import os
import yaml
from pathlib import Path
import shutil
from massibo_ana.preprocess.DataPreprocessor import DataPreprocessor
from massibo_ana.preprocess.NumpyDataPreprocessor import NumpyDataPreprocessor

# 1. Load the input parameters and change the WD to the workspace

In [ ]:
# Write here the ABSOLUTE path to the input parameters file
input_parameters_filepath = ''

In [ ]:
with open(
        input_parameters_filepath,
        "r",
        encoding="utf-8"
    ) as file:
        
        params = yaml.safe_load(file)

In [ ]:
# Run this one to load the actual workspace
root_directory = params['workspace_dir']
load_folderpath = root_directory + '/load'
aux_folderpath = root_directory + '/aux'
data_folderpath = root_directory + '/data'

In [ ]:
# Check set parameters
print(f"The pre-processed files will be placed in {root_directory}")

# 2. Check well-formed structure of measurements folder

In [ ]:
warnings, analysable_folderpaths = DataPreprocessor.check_structure_of_input_folder(
    params['input_data_folderpath'],
    subfolders_no=7,
    json_files_no_at_2nd_level=1,
    json_files_no_at_3rd_level=1,
    non_json_files_no_at_2nd_and_3rd_level=18
)

for warning in warnings:
    print('--> ' + warning)

# 3. Pre-process the data

In [ ]:
for folderpath in analysable_folderpaths:
    
    print('Preprocessing folder: ' + folderpath)
    print('-----------------------------------')

    if DataPreprocessor.hosts_gain_data(folderpath):
        json_default_values_filename = 'gainmeas_metadata.json'
    else:
        json_default_values_filename = 'darknoisemeas_metadata.json'

    path_to_json_default_values = os.path.join(
        folderpath, 
        json_default_values_filename
    )

    strips_ids = DataPreprocessor.grab_strip_IDs(
        path_to_json_default_values,
        3,
        verbose=params['verbose']
    )

    if params['data_from_tektronix_scope']:
        preprocessor_class = DataPreprocessor
        init_extra_kwargs = {
            'timestamp_prefix': 'ts_'
        }
        generate_meas_config_files_extra_kwargs = {
            'wvf_skiprows_identifier': 'TIME,'
        }
    else:
        preprocessor_class = NumpyDataPreprocessor

        aux, _ = DataPreprocessor.try_grabbing_from_json(
            {'packing_version': int},
            path_to_json_default_values,
            verbose=params['verbose']
        )

        try:
            packing_version = aux['packing_version']
        except KeyError:
            raise KeyError(
                f"Packing version not found in {path_to_json_default_values}"
            )

        init_extra_kwargs = {}
        generate_meas_config_files_extra_kwargs = {
            'packing_version': packing_version
        }

    mypreprocessor = preprocessor_class(
        folderpath,
        darknoise_base="W" if params['data_from_tektronix_scope'] else "W_",
        gain_base='G' if params['data_from_tektronix_scope'] else "G_",
        verbose=params['verbose'],
        **init_extra_kwargs
    )

    mypreprocessor.generate_meas_config_files(
        root_directory,
        load_folderpath,
        aux_folderpath,
        data_folderpath,
        path_to_json_default_values=path_to_json_default_values,
        sipms_per_strip=6,
        strips_ids=strips_ids,
        ask_for_inference_confirmation=False,
        verbose=params['verbose'],
        **generate_meas_config_files_extra_kwargs
    )

    print('-----------------------------------')
    print('\n\n', end='')

# 4. Delete the leftover files in the input folder

In [ ]:
for item in Path(params['input_data_folderpath']).iterdir():
    if item.is_file() or item.is_symlink():
        item.unlink()
    elif item.is_dir():
        shutil.rmtree(item)